In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from omegaconf import OmegaConf
import hydra
import torch

In [2]:
os.chdir('/home/sfp/streaming-flow-policy')
from diffusion_policy.dataset.franka_image_dataset import FrankaPickImageDataset
from diffusion_policy.policy.sfp_unet_hybrid_image_policy import SFPUnetHybridImagePolicy
from diffusion_policy.common.pytorch_util import dict_apply

/home/sfp/miniforge3/envs/robodiff2/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
dataset = FrankaPickImageDataset(
    horizon=17,
    max_train_episodes=90,
    pad_after=7,
    pad_before=1,
    seed=42,
    val_ratio=0.02,
    zarr_path='data/pushy_v3.zarr', #v3
    two_images=True
    )
train_dataloader = DataLoader(
    dataset, 
    batch_size=108,
    num_workers=8,
    persistent_workers=False,
    pin_memory=True,
    shuffle=True
    )
normalizer = dataset.get_normalizer()

Loading FrankaImageDataset from data/pushy_v3.zarr


In [4]:
cfg = OmegaConf.load("diffusion_policy/config/franka_sfp_image_pushy_2camera.yaml")
# cfg = OmegaConf.load("diffusion_policy/config/franka_sfp_image_pushy.yaml")

In [5]:
model: SFPUnetHybridImagePolicy = hydra.utils.instantiate(cfg.policy)
model.set_normalizer(normalizer)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['agent_pos']
using obs modality: rgb with keys: ['image', 'image2']
using obs modality: depth with keys: []
using obs modality: scan with keys: []


/home/sfp/miniforge3/envs/robodiff2/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/sfp/miniforge3/envs/robodiff2/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---------- Model Original -------------------
self.horizon 16
use_action_traj True
t span tensor([0.0000, 0.0625, 0.1250, 0.1875, 0.2500, 0.3125, 0.3750, 0.4375, 0.5000])
np.arange 0 8 1
select_action_indices [0 1 2 3 4 5 6 7]


In [6]:
obs_encoder = model.obs_encoder

In [7]:
for batch in train_dataloader:
    break

In [8]:
nobs = model.normalizer.normalize(batch['obs'])
this_nobs = dict_apply(nobs, 
            lambda x: x[:,:model.n_obs_steps,...].reshape(-1,*x.shape[2:]))
obs_encoder(this_nobs).shape

torch.Size([216, 74])

In [9]:
obs_encoder.output_shape()

[74]

In [ ]:
for k in batch:
    print(k)
print('---')
for k in batch['obs']:
    print(k)
print(batch['obs']['image'].shape)

obs
action
---
image
agent_pos


In [ ]:
model.compute_loss(batch, x_t_sigma=0.3)

tensor(0.2781, grad_fn=<MseLossBackward0>)

: 